In [1]:
# -*- coding:utf-8 -*-
# Modified Author: Inyong Hwang (inyong1020@gmail.com)
# Date: 2019-08-06-Tue
# 파이토치 첫걸음 Chapter 4. 이미지 처리와 합성곱 신경망

# 4.2 CNN을 사용한 이미지 분류

import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

from torchvision.datasets import FashionMNIST
from torchvision import transforms

fashion_mnist_train = FashionMNIST('./FashionMNIST',
                                   train=True,
                                   download=True,
                                   transform=transforms.ToTensor())
fashion_mnist_test = FashionMNIST('./FashionMNIST',
                                  train=False,
                                  download=True,
                                  transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=False)

In [2]:
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

net = nn.Sequential(
    conv_net,
    mlp
)

In [3]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

def train_net(net, train_loader, test_loader, 
              optimizer_cls = optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        net.train()
        n = 0
        n_acc = 0
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        
        val_acc.append(eval_net(net, test_loader, device))
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [4]:
net.to("cuda:0")

train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")

100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:09<00:00, 48.81it/s]


0 0.47218076238392764 0.8339333333333333 0.8763999938964844


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:08<00:00, 53.32it/s]


1 0.32157144625472206 0.8827833333333334 0.8953999876976013


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:08<00:00, 52.81it/s]


...

17 0.15955107200604218 0.94015 0.9181999564170837


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:08<00:00, 54.03it/s]


18 0.15767255892706478 0.9403333333333334 0.920799970626831


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:08<00:00, 53.90it/s]


19 0.15265515021597728 0.9422666666666667 0.911899983882904
